In [66]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA

In [67]:
# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
# Load the MobileNetV3 model
model = models.mobilenet_v3_small()
num_classes = 4  # Set this to your specific number of classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 
                                num_classes)
model.load_state_dict(torch.load(
    "D:/Projects/ML/GCN/Framework/cuda_tree_model/model_weights_v2"
    "/mobilenetv3_epoch_5.pth"))
model.eval()
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [69]:
# Modify model to extract features from the penultimate layer
feature_extractor = nn.Sequential(*list(model.children())[:-1]).to(device)

In [70]:
# Transform to match the input size for MobileNetV3
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [71]:
def read_annotations(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        label, x1, y1, x2, y2 = (parts[0], int(parts[1]), int(parts[2]), 
                                 int(parts[3]), int(parts[4]))
        if label.lower() == 'ld':
            label_id = 0
        elif label.lower() == 'hd':
            label_id = 1
        elif label.lower() == 'other':
            label_id = 2
        elif label.lower() == 'h':
            label_id = 3
        else:
            label_id = -1  # or some default value
        annotations.append((label_id, x1, y1, x2, y2))
    return annotations

In [72]:
def crop_image(image, bbox):
    x1, y1, x2, y2 = bbox
    return image[y1:y2+1, x1:x2+1]

In [73]:
image_folder = "D:/Projects/ML/GCN/Dataset_classifier/instance/images"
annotation_folder = "D:/Projects/ML/GCN/Dataset_classifier/instance/labels"
output_folder = "D:/Projects/ML/GCN/Outputs/features_mobilenetv3"
os.makedirs(output_folder, exist_ok=True)

In [74]:
def extract_features(image_path, annotations):
    image = cv2.imread(image_path)
    feature_vectors = []
    for annotation in annotations:
        label_id, x1, y1, x2, y2 = annotation
        cropped_image = crop_image(image, (x1, y1, x2, y2))
        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
        cropped_image_pil = Image.fromarray(cropped_image)
        input_tensor = preprocess(cropped_image_pil)
        input_batch = input_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            features = feature_extractor(input_batch)
        
        feature_vectors.append((label_id, x1, y1, x2, y2, 
                                features.squeeze().cpu().numpy()))
    return feature_vectors

In [75]:
def apply_pca(features, n_components=256):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(features)

In [76]:
# Collect all feature vectors for PCA
all_features = []

In [77]:
for annotation_file in os.listdir(annotation_folder):
    if annotation_file.endswith('.txt'):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, annotation_file)
        
        annotations = read_annotations(annotation_path)
        feature_vectors = extract_features(image_path, annotations)
        all_features.extend([fv[-1] for fv in feature_vectors])

In [78]:
all_features = np.array(all_features)
pca = PCA(n_components=256)
pca.fit(all_features)

PCA(n_components=256)

In [79]:
for annotation_file in os.listdir(annotation_folder):
    if annotation_file.endswith('.txt'):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, annotation_file)
        
        annotations = read_annotations(annotation_path)
        feature_vectors = extract_features(image_path, annotations)
        
        output_annotations = []
        for label_id, x1, y1, x2, y2, features in feature_vectors:
            reduced_features = pca.transform([features])[0]
            feature_str = ' '.join(map(str, reduced_features))
            output_annotations.append(
                f"{label_id} {x1} {y1} {x2} {y2} {feature_str}")
        
        output_annotation_path = os.path.join(output_folder, annotation_file)
        with open(output_annotation_path, 'w') as out_file:
            out_file.write('\n'.join(output_annotations))